In [ ]:
import re
import csv

Код в клетках ниже позволяет собрать подкорпус из предложений, в которых встретился глагол падения из заданного списка

In [ ]:
verbs = 'падать, упасть, выпасть, опасть, завалиться, повалиться, отвалиться, обвалиться, сорваться, отскочить, соскочить,рухнуть, обрушиться, сыпаться, высыпаться, осыпаться, шлепнуться, грохнуться, шмякнуться, брякнуться, бултыхнуться, плюхнуться'
list_verbs = verbs.split(', ')

In [ ]:
target_verbs = ['\t'+verb for verb in list_verbs]

In [ ]:
current_sentence = ''

with open('lines_3000000.txt',  encoding = 'utf-8') as f:
    for line in f:            
        if 'SENT' in line:
            current_sentence +='\n\n'
            if any(verb in current_sentence for verb in target_verbs):
                with open('subcorpus.txt', 'a', encoding = 'utf-8') as file:
                    file.write(current_sentence)
            current_sentence = ''
        else:
            current_sentence += line    

Открыть файл с подкорпусом, он окажется в формате list, каждый элемент - предложение

In [ ]:
with open ('subcorpus.txt', 'r', encoding = 'utf-8') as f:
    content = f.read()
    sentences = content.split('\n\n\n')  

Код ниже собирает статистику по вхождению каждого глагола в корпус

In [ ]:
for s in sentences:
    tokens = [token.split('\t') for token in s.split('\n')] 
    for line in tokens:
        
        if any(verb in line for verb in target_verbs):
            target_line = line
            target_verb = target_line[2]
            
            verb_stats[target_verb]+=1

In [ ]:
with open ('verb_stats.txt', 'a', encoding ='utf-8') as s:
    for k in verb_stats:
        s.write(k+'\t'+str(verb_stats[k])+'\n')

Следующие клетки решают задачу сбора информации об аргументной структуре каждого глагола

In [ ]:
for s in sentences[id_start:]:
    tokens = [token.split('\t') for token in s.split('\n')] 
    id = tokens[0][0]
    #тут делим на строки(слова) + тэги
    counter = 1
    for line in tokens:

        if any(verb in line for verb in target_verbs):
            target_line = line
            target_verb = target_line[2]
            target_line_id = len(tokens) - counter - 1
            
            verb_stats[target_verb]+=1
            try:
                args = [k for k in tokens[1:] if k[5]==target_line[4]]         
            except IndexError:
                continue
        else:
            counter+=1
    
    if any('предик' in arg for arg in args) == False:
        tokens.reverse()

        
        for line in tokens[target_line_id:target_line_id+7]:
            print(line)
            if len(line) == 7:
                if line[3] in ['N', 'P'] and line[1][:-3:-1] in ['nn', 'yn'] and line[6] in ['предик', 'аппоз']:
                    subject = '_'.join(line)
                    print(subject)
                    args.append(subject)
                    break
            else:
                continue  
                
    with open ('argstructure_new.txt', 'a', encoding='utf-8') as af:
        if len(args) != 0:
            arguments = '\t\t'.join(list(['\t'.join(a) for a in args])) 
            af.write(str(id)+'\n'+target_verb+'\n'+arguments+'\n\n')
        else:
            af.write(str(id)+'\n'+target_verb+'\n'+'no arguments'+'\n\n')     

С помощью файла с аргументной структурой, файл ниже собирает информацию об отдельных вхождениях в заданных предложениях (по id)(в нашем случае для конкретных 100 примеров в одном фрейме)

Итоговый файл содержит информацию об аргументах глагола в формате:

глагол, id предложения, аргумент1|аргумент2|аргумент3

In [1]:
with open ('argstructure_new.txt', 'r', encoding = 'utf-8') as ar:
    content = ar.read()
    
items = content.split('\n\n')

In [ ]:
suitable = ['1-компл', 'предик', '2-компл', 'обст', 'аппоз' ]


with open('frame1.csv', "w", newline='', encoding = 'utf-8') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(['verb', 'id', 'args'])

    for id in ids:
        entry_info = items[int(id)]
        ID, verb, args = entry_info.split('\n')
        args_splitted = [a.split('\t') for a in args.split('\t\t')]
        clean_args = [arg for arg in args_splitted if any(pos in arg for pos in suitable)]
        nargs = ['\t'.join(k) for k in clean_args]
        kargs = '\t\t'.join(nargs)
        normal_args = re.sub('\t\t', '|', kargs)
        norm_args2 = re.sub('\t', '_', normal_args)
        info = [verb, str(id), norm_args2] 
        
        writer.writerow(info)

Разметка предложений на предмет наличия субъекта, начальной точки, конечной точки

Итоговый файл содержит следуюущую информацию:
    
глагол, id предложения, наличие субъекта (0, 1), информация о субъекте, наличие нач.точки(0, 1), информация о нач.точке, наличие кон.точки(0, 1), информация о кон.точке 

In [ ]:
csv_file1  =  open('frame1_ANNO.csv', "w", newline='', encoding = 'utf-8')
writer = csv.writer(csv_file1, delimiter=',')

with open('frame1.csv', encoding = 'utf-8') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0

    
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            verb = row[0]
            id = row[1]
            args = row[2].split('|')
            
            #print(verb, id, args)
            
            subject = 0
            subject_info = 0
            goal = 0
            goal_info = 0
            source = 0
            source_info = 0
            
            sentence = stcs_nod_noid[int(id)].split('\n')
            
            for arg in args:
                
                print(arg)
                
                ###СУБЪЕКТ
                try:
                    if 'предик' in arg or 'аппоз' in arg:
                        subject  = 1
                        subject_info = arg
                    
                    if 'компл' in arg or 'обст' in arg:
                        if arg.split('_')[3] == 'S':
                            prep = arg.split('_')[0]
                        #print(sentence)
                            noun = sentence[int(arg.split('_')[4])]
                            print(noun)
                        
                            poss = ["\tA\t", "\tV\t", "\tP\t", 'PUNC', "\tM\t"]
                            shift = 1
                        
                            if int(arg.split('_')[4]) < len(sentence)-1:
                                if  any(pos in noun for pos in poss):
                                    try:
                                        while "\tN\t" not in noun and 'gn\t' not in noun:
                                            print ('NOT NOUN ')
                                            noun = sentence[int(arg.split('_')[4])+shift]
                                            print(noun)
                                            shift += 1
                                    except IndexError:
                                        pass
                                
                            ppnoun = re.sub('\t', '_', noun)
                        #print(ppnoun)
                        
                            if ppnoun.split('_')[1][-2:] in ['an', 'ay'] and goal == 0:
                                goal = 1
                                goal_info = arg+'\t\t'+ppnoun
                                print('GOAL')
                            elif ppnoun.split('_')[1][-2:]== 'gn' and prep.lower() in ['из', 'со', 'с', 'из-под' ]  and source == 0:
                                source = 1
                                source_info = arg+'\t\t'+ppnoun
                                print('SOURCE')
                            
                        elif arg.split('_')[3] == 'R':
                            if arg.split('_')[0].lower() =='сверху':
                                source = 1
                                source_info = re.sub('\t', '_', arg)
                                print('SOURCE')
                            elif arg.split('_')[0].lower() in ['вниз', 'вбок']:
                                goal = 1
                                goal_info = re.sub('\t', '_', arg)
                                print('GOAL')
                            
                        elif arg.split('_')[3] == 'P':
                            if arg.split('_')[0].lower() =='откуда':
                                source = 1
                                source_info = re.sub('\t', '_', arg)
                                print('SOURCE')
                            elif arg.split('_')[0].lower() =='куда':
                                goal = 1
                                goal_info = re.sub('\t', '_', arg)
                                print('GOAL')
                except IndexError:
                    pass
                
            line_count += 1
        
                  
            writer.writerow([verb, id, subject, subject_info, source, source_info, goal, goal_info])
        print(line_count)
            
csv_file1.close() 

                      
                      